In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *


import os
from sklearn.model_selection import KFold


In [2]:
train = pd.read_pickle('e:/kaggle/09504/train_scaled.pkl')
test = pd.read_pickle('e:/kaggle/09504/test_scaled.pkl')
y = pd.read_pickle('e:/kaggle/09504/y.pkl')

In [3]:
from BayDS.lib.training import *
from category_encoders.cat_boost import CatBoostEncoder

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [4]:
test.columns

Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       'Transaction_freq_2d_past', 'Transaction_freq_3d_past',
       'Transaction_freq_7d_past', 'Transaction_freq_30d_past',
       'Transaction_freq_5_past', 'Transaction_freq_5_center',
       'Transaction_freq_10_past', 'Transaction_freq_10_center',
       'Transaction_freq_100_past', 'Transaction_freq_100_center'],
      dtype='object', length=698)

In [5]:
import yaml
cat_col = yaml.load(open(r'e:/kaggle/09504/categorical_columns.yaml','r'),Loader=yaml.FullLoader) 
num_col = yaml.load(open(r'e:/kaggle/09504/numerical_columns.yaml','r'),Loader=yaml.FullLoader) 


In [6]:
undef_cols = []
for col in test.columns:
    if col not in cat_col and col not in num_col:
        print (col)
#         undef_cols.append(col)
# yaml.dump( undef_cols,open('e:/kaggle/09504/undef_cols.yaml','w') )
        

In [7]:
# from sklearn.preprocessing import StandardScaler
# for column in num_col:
#     print(column)
#     if column not in train.columns:
#         print("Absent column ", column)
#         continue
#     col = pd.concat([train[column],test[column]]).astype(np.float32)
#     filtered = col[~col.isin([np.inf, -np.inf])].dropna()
#     vmax = filtered.max()
#     vmin = filtered.min()
#     vmean = filtered.mean()
#     col = col.replace({
#         np.inf:vmax,
#         -np.inf:vmin
#        }).fillna(vmean)
#     if vmax > 100 and vmin >= 0:
#         col = np.log1p(col)
    
#     scaler = StandardScaler()
#     col.values[...] = scaler.fit_transform(col.values.reshape(-1,1)).flatten()
    
#     train[column] = col[train.index]
#     test[column] = col[test.index]


In [8]:
# train.to_pickle('e:/kaggle/09504/train_scaled.pkl')
# test.to_pickle('e:/kaggle/09504/test_scaled.pkl')

In [9]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)

In [10]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[train.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits
    x = Dense(600, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(300, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.3)(x)
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8000,
    'epochs': 30,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [11]:
gc.collect()
result_dict_keras = train_model_classification_vb(model=NNModel_maker, 
                                             X=train,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             categorial_columns=cat_col, categorial_encoder=CatBoostEncoder,
                                             groups=None)

Fold 1 started at Sat Sep 28 22:32:44 2019
Encoding categorical features




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 472432 samples, validate on 118108 samples
Epoch 1/30
472432/472432 [==============================] - 32s 68us/step - loss: 0.1920 - val_loss: 0.1215
Epoch 2/30
472432/472432 [==============================] - 25s 52us/step - loss: 0.1233 - val_loss: 0.1200
Epoch 3/30
472432/472432 [==============================] - 24s 51us/step - loss: 0.1146 - val_loss: 0.1138
Epoch 4/30
472432/472432 [==============================] - 22s 48us/step - loss: 0.1075 - val_loss: 0.1112
Epoch 5/30
472432/472432 [==============================] - 24s 51us/step - loss: 0.1013 - val_loss: 0.1086
Epoch 6/30
472432/472432 [==============================] - 24s 52us/step - loss: 0.0929 - val_loss: 0.1030
Epoch 7/30
4

472432/472432 [==============================] - 21s 46us/step - loss: 0.0577 - val_loss: 0.0876
Epoch 28/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.0574 - val_loss: 0.0894
Epoch 29/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0571 - val_loss: 0.0883
Epoch 30/30
118108/118108 [==============================] - 1s 10us/step
Fold 1. auc: 0.9135.
Fold 3 started at Sat Sep 28 23:00:04 2019
Encoding categorical features
Train on 472432 samples, validate on 118108 samples
Epoch 1/30
472432/472432 [==============================] - 23s 48us/step - loss: 0.1759 - val_loss: 0.1210
Epoch 2/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.1158 - val_loss: 0.1138
Epoch 3/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.1088 - val_loss: 0.1086
Epoch 4/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.1032 - val_loss: 0.1047
Epoch 5/30
472432/472432 [=======

472432/472432 [==============================] - 22s 46us/step - loss: 0.0864 - val_loss: 0.0915
Epoch 7/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0804 - val_loss: 0.0895
Epoch 8/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0768 - val_loss: 0.0888
Epoch 9/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0731 - val_loss: 0.0890
Epoch 10/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0720 - val_loss: 0.0891
Epoch 11/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0697 - val_loss: 0.0879
Epoch 12/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0690 - val_loss: 0.0869
Epoch 13/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0673 - val_loss: 0.0873
Epoch 14/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.0665 - val_loss: 0.0868
Epoch 15/30
472432/472432 [=======

In [14]:
import pickle
with open(f'e:/kaggle/09504/result_dict_keras', 'wb') as f:
    #     q = json.dumps(result_dict_lgb,indent=2)
        pickle.dump(result_dict_keras,f)

In [ ]:

# result_timefreq = train_model_classification_vb(model=NNModel_maker,
#                                              X=train, X_test=test, y=y, params=params, folds=folds,
#                                              model_type=train_options['model_type'],
#                                              eval_metric=train_options['eval_metric'],
#                                              plot_feature_importance=True,
#                                              verbose=500, 
                                             
#                                              averaging=train_options['averaging'],
#                                              n_jobs=-1, groups=None,
# #                                              train_1_sample_coef=None, train_0_sample_coef=0.2,
# #                                              categorial_columns=cat_col, categorial_encoder=CatBoostEncoder
#                                                )

# In[50]:


pd.DataFrame(result_dict_keras['oof'], columns=['isFraud'], index=train.index).to_csv(
    'oof_all_data_best_lgb_timefreq.csv')
pd.DataFrame(result_dict_keras['prediction'], columns=['isFraud'], index=test.index).to_csv(
    'prediction_all_data_best_lgb_timefreq.csv')

sample_submission = pd.read_csv(data_path + 'sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict_keras['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1  # *0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv('lgb_timefreq.csv')
